In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

nltk.download('punkt')
nltk.download('wordnet')

# Cargar datos desde el archivo JSON
with open('datos.json') as file:
    datos = json.load(file)

# Preprocesar los datos
lemmatizer = WordNetLemmatizer()

palabras = []
tags = []
intents = datos['intents']

for intent in intents:
    for pattern in intent['patterns']:
        # Tokenizar y lematizar las palabras
        words = nltk.word_tokenize(pattern)
        palabras.extend(words)
        tags.append(intent['tag'])

palabras = [lemmatizer.lemmatize(word.lower()) for word in palabras if word.isalnum()]
palabras = sorted(list(set(palabras)))

tags = sorted(list(set(tags)))

# Cargar el modelo entrenado
modelo = load_model('chatbot_modelo.h5')

# Función para procesar el mensaje del usuario y obtener una respuesta del chatbot
def procesar_mensaje(mensaje):
    # Bolsa de palabras para el mensaje del usuario
    bolsa_palabras = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(mensaje) if word.isalnum()]

    # Codificación del mensaje en una matriz de entrada
    entrada_usuario = [1 if palabra in bolsa_palabras else 0 for palabra in palabras]

    # Predicción del modelo
    resultado = modelo.predict(np.array([entrada_usuario]))

    # Obtener la etiqueta predicha
    predicha_tag = tags[np.argmax(resultado)]

    # Buscar la respuesta asociada a la etiqueta
    for intent in intents:
        if intent['tag'] == predicha_tag:
            respuesta = random.choice(intent['responses'])
            return respuesta

# Bucle de conversación
print("¡Hola! Soy un chatbot. Puedes escribir 'salir' para terminar la conversación.")

while True:
    # Obtener mensaje del usuario
    mensaje_usuario = input("Usuario: ")

    # Salir si el usuario escribe "salir"
    if mensaje_usuario.lower() == 'salir':
        print("¡Hasta luego!")
        break

    # Procesar el mensaje y obtener la respuesta del chatbot
    respuesta_chatbot = procesar_mensaje(mensaje_usuario)
    print("Chatbot:", respuesta_chatbot)


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

nltk.download('punkt')
nltk.download('wordnet')

# Cargar datos desde el archivo JSON
with open('datos.json') as file:
    datos = json.load(file)

# Preprocesar los datos
lemmatizer = WordNetLemmatizer()

palabras = []
tags = []
intents = datos['intents']

for intent in intents:
    for pattern in intent['patterns']:
        # Tokenizar y lematizar las palabras
        words = nltk.word_tokenize(pattern)
        palabras.extend(words)
        tags.append(intent['tag'])

palabras = [lemmatizer.lemmatize(word.lower()) for word in palabras if word.isalnum()]
palabras = sorted(list(set(palabras)))

tags = sorted(list(set(tags)))

# Crear conjuntos de datos de entrada y salida
entrenamiento = []
salida_vacia = [0] * len(tags)

for intent in intents:
    for pattern in intent['patterns']:
        # Bolsa de palabras para cada patrón
        bolsa_palabras = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(pattern) if word.isalnum()]

        # Codificación de un patrón en una matriz de entrada
        fila_entrenamiento = [1 if palabra in bolsa_palabras else 0 for palabra in palabras]

        # Codificación de la etiqueta en una matriz de salida
        etiqueta = list(salida_vacia)
        etiqueta[tags.index(intent['tag'])] = 1

        entrenamiento.append([fila_entrenamiento, etiqueta])

random.shuffle(entrenamiento)
entrenamiento = np.array(entrenamiento)

# Separar datos de entrada y salida
X_train = list(entrenamiento[:, 0])
y_train = list(entrenamiento[:, 1])

# Construir el modelo de red neuronal
modelo = Sequential()
modelo.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(64, activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(len(y_train[0]), activation='softmax'))

# Compilar el modelo
modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

for _ in range(5):
    ##modelo.fit(np.array(X_train), np.array(y_train), epochs=1, batch_size=5, verbose=1)

# Entrenar el modelo
    historial_entrenamiento = modelo.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=5, verbose=1)

# Guardar el modelo entrenado
modelo.save('chatbot_modelo.h5')
print("Modelo entrenado y guardado exitosamente.")


# Función para procesar el mensaje del usuario y obtener una respuesta del chatbot
def procesar_mensaje(mensaje):
    # Bolsa de palabras para el mensaje del usuario
    bolsa_palabras = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(mensaje) if word.isalnum()]

    # Codificación del mensaje en una matriz de entrada
    entrada_usuario = [1 if palabra in bolsa_palabras else 0 for palabra in palabras]

    # Predicción del modelo
    resultado = modelo.predict(np.array([entrada_usuario]))

    # Obtener la etiqueta predicha
    predicha_tag = tags[np.argmax(resultado)]

    # Buscar la respuesta asociada a la etiqueta
    for intent in intents:
        if intent['tag'] == predicha_tag:
            respuesta = random.choice(intent['responses'])
            return respuesta

# Bucle de conversación
print("¡Hola! Soy un chatbot. Puedes escribir 'salir' para terminar la conversación.")

while True:
    # Obtener mensaje del usuario
    mensaje_usuario = input("Usuario: ")

    # Salir si el usuario escribe "salir"
    if mensaje_usuario.lower() == 'salir':
        print("¡Hasta luego!")
        break

    # Procesar el mensaje y obtener la respuesta del chatbot
    respuesta_chatbot = procesar_mensaje(mensaje_usuario)
    print("Chatbot:", respuesta_chatbot)
